# Sentiment Analysis (Deep Learning, LSTM)

In this tutorial, we perform sentiment analysis using deep learning, where we use a Recurrent Neural Network (RNN) network structure. More specifically, we perform a many a many-to-one classification -- that is, given a sequence of words, we are only interested in one final class (not, e.g., into a class for each word in sequence):

<img src="images/rnn-many-to-one-sequence-classification.jpg" alt="Drawing" style="width: 20%;"/>


## Import required packages

In [1]:
import os
import numpy as np
import pandas as pd
import csv

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, Embedding, Dropout, Bidirectional

from sklearn.preprocessing import LabelBinarizer

# The next imports are only needed for the preprocessing
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from utils.nlputil import preprocess_text

Using TensorFlow backend.


In [2]:
tweet_tokenizer = TweetTokenizer()
wordnet_lemmatizer = WordNetLemmatizer()

In [3]:
NUM_LABELS = 3 # We have 3 polarity classes
MAX_WORDS = 1000
EMBEDDING_DIM = 50
LSTM_HIDDEN_DIM = 32

## Date preparation

### Load data from files

In [4]:
df_tweets_train = pd.read_csv('data/twitter-sentiment/twitter-sentiment-bowden-training.csv')

# Print the first 5 lines
df_tweets_train.head()

,tweet,senti
0,@united UA5396 can wait for me. I'm on the gro...,0
1,I hate Time Warner! Soooo wish I had Vios. Can...,0
2,Tom Shanahan's latest column on SDSU and its N...,2
3,Found the self driving car!! /IWo3QSvdu2,2
4,@united arrived in YYZ to take our flight to T...,0


### Preprocess training and test data

In [5]:
train_tweets = df_tweets_train['tweet']
train_polarities = df_tweets_train['senti']

train_tweets_processed = [''] * len(train_tweets)

for idx, doc in enumerate(train_tweets):
    train_tweets_processed[idx] = preprocess_text(doc, tokenizer=tweet_tokenizer, lemmatizer=wordnet_lemmatizer)

In [6]:
df_tweets_test = pd.read_csv('data/twitter-sentiment/twitter-sentiment-bowden-test.csv')

test_tweets = df_tweets_test['tweet']
test_polarities = df_tweets_test['senti']  

test_tweets_processed = [''] * len(test_tweets)

for idx, doc in enumerate(test_tweets):
    test_tweets_processed[idx] = preprocess_text(doc, tokenizer=tweet_tokenizer, lemmatizer=wordnet_lemmatizer)  

### Prepare labels

In [7]:
encoder = LabelBinarizer()
encoder.fit(train_polarities)
y_train = encoder.transform(train_polarities)
y_test = encoder.transform(test_polarities)

print(y_test[:10])

[[1 0 0]
 [1 0 0]
 [1 0 0]
 [0 0 1]
 [1 0 0]
 [1 0 0]
 [0 0 1]
 [1 0 0]
 [0 0 1]
 [0 1 0]]


### Calculate maximum sequence length 

Most neural networks assume inputs of the same size. Since we are using tweets here which are usually rather short, we can find the longest one (in terms of the number of words) and define its length as the maximum sequence length. In case of longer texts, e.g., reviews, the maximum sequence length is specified a priori to typically a couple of hundred.

In [8]:
longest_train_tweet = max([len(s.split()) for s in train_tweets_processed])
longest_test_tweet = max([len(s.split()) for s in test_tweets_processed])

max_seq_len = max(longest_train_tweet, longest_test_tweet)

print("Maximum sequence length: {}".format(max_seq_len))

Maximum sequence length: 29


In [9]:
tokenizer = Tokenizer(num_words=MAX_WORDS)
tokenizer.fit_on_texts(train_tweets_processed)

In [10]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 2709 unique tokens.


### Convert strings to sequences

The `tokenizer.word_index` each word in the vocabulary to an index. The method `texts_to_sequences` now converts a string into a list of indexes representing the words in the string

In [11]:
X_train = tokenizer.texts_to_sequences(train_tweets_processed)
X_test = tokenizer.texts_to_sequences(test_tweets_processed)

max_idx = max([ max(l) for l in X_train if len(l) > 0])

print(X_train[0])
print("Largest used index: {}".format(max_idx)) # This should be (MAX_WORDS-1)


[4, 838, 50, 8, 498, 174, 6, 200, 337, 499, 500]
Largest used index: 999


### Sequence padding.

We have to ensure that all inputs have the same length. Above, we calculated the maximum length being 29. That means, we have to "pad" all tweets that are shorter than that. Keras comes with a handy method for that. `padding='post'` specifies that the padding is done after the last wors. `truncating='post'` is not required in this example; it would cut of words from then end tweets that are too long (which cannot happen here).

In [12]:
X_train = pad_sequences(X_train, maxlen=max_seq_len, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_seq_len, padding='post', truncating='post')

print(X_train[0])
print("Sequence length: {}".format(len(X_train[0])))

[  4 838  50   8 498 174   6 200 337 499 500   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0]
Sequence length: 29


## Training the model (with word embeddings)

### Define network model

In [13]:
model = Sequential()

model.add(Embedding(MAX_WORDS, EMBEDDING_DIM, input_length=max_seq_len))
model.add(Bidirectional(LSTM(LSTM_HIDDEN_DIM)))
model.add(Dense(128))
model.add(Activation('relu'))
#model.add(Dense(64)) # Optional
#model.add(Activation('relu')) # Optional
model.add(Dense(NUM_LABELS))
model.add(Activation('softmax'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 29, 50)            50000     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                21248     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
activation_1 (Activation)    (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 387       
_________________________________________________________________
activation_2 (Activation)    (None, 3)                 0         
Total params: 79,955
Trainable params: 79,955
Non-trainable params: 0
_________________________________________________________________
None


### Compile model

In [14]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### Train model

In [15]:
history = model.fit(X_train, y_train, batch_size=32, epochs=100, verbose=1, validation_split=0.1)

Train on 629 samples, validate on 70 samples
Epoch 1/100
629/629 [==============================] - 3s 5ms/step - loss: 1.0867 - acc: 0.3990 - val_loss: 1.0853 - val_acc: 0.4429
Epoch 2/100
629/629 [==============================] - 1s 1ms/step - loss: 1.0508 - acc: 0.4913 - val_loss: 1.0490 - val_acc: 0.5429
Epoch 3/100
629/629 [==============================] - 1s 1ms/step - loss: 0.9012 - acc: 0.6868 - val_loss: 0.9163 - val_acc: 0.5429
Epoch 4/100
629/629 [==============================] - 1s 1ms/step - loss: 0.5661 - acc: 0.7838 - val_loss: 0.7700 - val_acc: 0.6429
Epoch 5/100
629/629 [==============================] - 1s 1ms/step - loss: 0.3265 - acc: 0.8967 - val_loss: 0.8135 - val_acc: 0.6571
Epoch 6/100
629/629 [==============================] - 1s 1ms/step - loss: 0.1782 - acc: 0.9571 - val_loss: 0.9742 - val_acc: 0.7000
Epoch 7/100
629/629 [==============================] - 1s 1ms/step - loss: 0.1004 - acc: 0.9777 - val_loss: 1.0705 - val_acc: 0.7286
Epoch 8/100
629/629 [===

### Evaluate model

In [16]:
score = model.evaluate(X_test, y_test, batch_size=32, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

298/298 [==============================] - 0s 215us/step
Test score: 2.4030683824679997
Test accuracy: 0.7013422814791634
